In [1]:
import os
import pandas as pd
import yaml
import copy
import random
from pathlib import Path
import matplotlib.pyplot as plt

import aot


In [2]:
base_dir = Path(aot.__path__[0])
core_expt_yaml_path = base_dir / "experiment/core_exp_settings.yml"
stimuli_temp_path = base_dir / "experiment/stimuli_settings_temp.yml"
core_settings = yaml.load(open(core_expt_yaml_path), Loader=yaml.FullLoader)
stimuli_settings_temp = yaml.load(open(stimuli_temp_path), Loader=yaml.FullLoader)
total_video_number = core_settings["various"]["total_video_number"]
settings_root_path = base_dir / core_settings["paths"]["settings_path"]
main_settings_path = settings_root_path / "main"

video_names = []
for i in range(1, 1 + total_video_number):
    # 0 padding for video name
    video_name = str(i).zfill(4) + ".mp4"
    video_names.append(video_name)
print("videos number: ", len(video_names))

origin_videos_number = len(video_names)
subject_number = core_settings["various"]["subject_number"]
print("subject number: ", subject_number)
session_number = core_settings["various"]["session_number"]
print("session number: ", session_number)
run_number = core_settings["various"]["run_number"]
print("run number: ", run_number)
# only for main run
blank_number_per_run = core_settings["various"]["blank_number_per_run"]
unique_video_number_per_session = core_settings["various"][
    "unique_video_number_per_session"
]
video_number_per_run_main = int((unique_video_number_per_session * 4) / run_number)
# only 1 subject and 1 session for grading
video_number_per_run_grading = int(origin_videos_number / run_number)

videos number:  2179
subject number:  6
session number:  12
run number:  10


In [3]:
def read_settings_file(subject, ses, run):
    settings_file = main_settings_path / f"experiment_settings_sub_{str(subject).zfill(2)}_ses_{str(ses).zfill(2)}_run_{str(run).zfill(2)}.yml"
    settings = yaml.load(open(settings_file), Loader=yaml.FullLoader)
    return settings

In [4]:
def read_settings_files_session(subject, ses):#return a list of settings files for a given session
    settings_files = [read_settings_file(subject, ses, run) for run in range(1,run_number+1)]
    sessions_movies = [settings['stimuli']['movie_files'] for settings in settings_files]
    return sessions_movies

In [5]:
def check_video_replay_of_session(session_movies):
    #check if the same video is replayed in a session
    #append all the lists of movies in a session
    all_movies = []
    for run_movies in session_movies:
        all_movies += run_movies
    #remove the blanks
    all_movies = [movie for movie in all_movies if movie != 'blank'] 
    #check if there is a replay
    stack = []
    for movie in all_movies:
        if movie in stack:
            stack.remove(movie)
        else:
            stack.append(movie)
    return len(stack) == 0

def check_video_replay_all_session():
    for subject in range(1, subject_number+1):
        for ses in range(1, session_number+1):
            session_movies = read_settings_files_session(subject, ses)
            #print(f"subject {subject}, session {ses} movies: {session_movies}")
            is_replay = check_video_replay_of_session(session_movies)
            print(f"subject {subject}, session {ses} replay: {is_replay}")

check_video_replay_all_session()

subject 1, session 1 replay: True
subject 1, session 2 replay: True
subject 1, session 3 replay: True
subject 1, session 4 replay: True
subject 1, session 5 replay: True
subject 1, session 6 replay: True
subject 1, session 7 replay: True
subject 1, session 8 replay: True
subject 1, session 9 replay: True
subject 1, session 10 replay: True
subject 1, session 11 replay: True
subject 1, session 12 replay: True
subject 2, session 1 replay: True
subject 2, session 2 replay: True
subject 2, session 3 replay: True
subject 2, session 4 replay: True
subject 2, session 5 replay: True
subject 2, session 6 replay: True
subject 2, session 7 replay: True
subject 2, session 8 replay: True
subject 2, session 9 replay: True
subject 2, session 10 replay: True
subject 2, session 11 replay: True
subject 2, session 12 replay: True
subject 3, session 1 replay: True
subject 3, session 2 replay: True
subject 3, session 3 replay: True
subject 3, session 4 replay: True
subject 3, session 5 replay: True
subject 

In [6]:
def check_video_direction_duality_of_session(session_movies):
    #check whether the videos in a session are duality pairs(R_S_movie,S_movie)
    #append all the lists of movies in a session
    def check_duality(video1,video2):
        '''
        if video1.startswith('R_') and video2.startswith('S_'):
            return video1[4:] == video2[2:]
        elif video1.startswith('S_') and video2.startswith('R_'):
            return video1[2:] == video2[4:]
        else:
            return False
        '''
        if video1.endswith('_fw.mp4') and video2.endswith('_rv.mp4'):
            return video1[:-7] == video2[:-7]
        elif video1.endswith('_rv.mp4') and video2.endswith('_fw.mp4'):
            return video1[:-7] == video2[:-7]
        else:
            return False
    def is_duality_in_stack(video, video_stack):#with side effect that remove the duality pair
        for v in video_stack:
            if check_duality(video,v):
                video_stack.remove(v)
                return True
        return False
    all_movies = []
    for run_movies in session_movies: 
        all_movies += run_movies
    #remove the blanks
    all_movies = [movie for movie in all_movies if movie != 'blank']
    stack = []
    for movie in all_movies:
        if not is_duality_in_stack(movie, stack):
            stack.append(movie)
    return len(stack) == 0

def check_video_direction_duality_all_session():
    for subject in range(1, subject_number+1):
        for ses in range(1, session_number+1):
            session_movies = read_settings_files_session(subject, ses)
            #print(f"subject {subject}, session {ses} movies: {session_movies}")
            is_duality = check_video_direction_duality_of_session(session_movies)
            print(f"subject {subject}, session {ses} duality: {is_duality}")

check_video_direction_duality_all_session()
    

subject 1, session 1 duality: True
subject 1, session 2 duality: True
subject 1, session 3 duality: True
subject 1, session 4 duality: True
subject 1, session 5 duality: True
subject 1, session 6 duality: True
subject 1, session 7 duality: True
subject 1, session 8 duality: True
subject 1, session 9 duality: True
subject 1, session 10 duality: True
subject 1, session 11 duality: True
subject 1, session 12 duality: True
subject 2, session 1 duality: True
subject 2, session 2 duality: True
subject 2, session 3 duality: True
subject 2, session 4 duality: True
subject 2, session 5 duality: True
subject 2, session 6 duality: True
subject 2, session 7 duality: True
subject 2, session 8 duality: True
subject 2, session 9 duality: True
subject 2, session 10 duality: True
subject 2, session 11 duality: True
subject 2, session 12 duality: True
subject 3, session 1 duality: True
subject 3, session 2 duality: True
subject 3, session 3 duality: True
subject 3, session 4 duality: True
subject 3, ses

In [7]:
def count_same_video_in_one_run(subject, ses, run):
    #count the number of same video in one run
    settings = read_settings_file(subject, ses, run)
    movies = settings['stimuli']['movie_files']
    movies = [movie for movie in movies if movie != 'blank']
    count = 0
    stack = []
    for movie in movies:
        if movie in stack:
            stack.remove(movie)
            count += 1
        else:
            stack.append(movie)
    return count

def count_same_video_in_all_run():
    count_list = []
    for subject in range(1, subject_number+1):
        for ses in range(1, session_number+1):
            for run in range(1,run_number+1):
                count = count_same_video_in_one_run(subject, ses, run)
                count_list.append(count)
                print(f"subject {subject}, session {ses} ,run {run} same video count: {count}")
    return count_list

count_same_video_in_all_run()


subject 1, session 1 ,run 1 same video count: 2
subject 1, session 1 ,run 2 same video count: 6
subject 1, session 1 ,run 3 same video count: 3
subject 1, session 1 ,run 4 same video count: 3
subject 1, session 1 ,run 5 same video count: 2
subject 1, session 1 ,run 6 same video count: 2
subject 1, session 1 ,run 7 same video count: 3
subject 1, session 1 ,run 8 same video count: 5
subject 1, session 1 ,run 9 same video count: 5
subject 1, session 1 ,run 10 same video count: 7
subject 1, session 2 ,run 1 same video count: 2
subject 1, session 2 ,run 2 same video count: 5
subject 1, session 2 ,run 3 same video count: 4
subject 1, session 2 ,run 4 same video count: 4
subject 1, session 2 ,run 5 same video count: 1
subject 1, session 2 ,run 6 same video count: 2
subject 1, session 2 ,run 7 same video count: 4
subject 1, session 2 ,run 8 same video count: 3
subject 1, session 2 ,run 9 same video count: 3
subject 1, session 2 ,run 10 same video count: 5
subject 1, session 3 ,run 1 same video

[2,
 6,
 3,
 3,
 2,
 2,
 3,
 5,
 5,
 7,
 2,
 5,
 4,
 4,
 1,
 2,
 4,
 3,
 3,
 5,
 6,
 5,
 6,
 5,
 2,
 4,
 5,
 3,
 2,
 3,
 5,
 5,
 2,
 5,
 4,
 2,
 3,
 3,
 6,
 3,
 3,
 2,
 4,
 5,
 3,
 4,
 2,
 3,
 6,
 4,
 3,
 3,
 3,
 1,
 5,
 4,
 4,
 3,
 4,
 2,
 3,
 7,
 5,
 1,
 3,
 4,
 6,
 3,
 4,
 2,
 3,
 3,
 7,
 1,
 5,
 6,
 0,
 4,
 5,
 6,
 5,
 4,
 4,
 4,
 5,
 5,
 3,
 2,
 4,
 4,
 3,
 2,
 2,
 0,
 5,
 5,
 6,
 2,
 3,
 4,
 5,
 3,
 2,
 6,
 3,
 4,
 4,
 7,
 7,
 4,
 4,
 7,
 4,
 3,
 6,
 6,
 1,
 6,
 2,
 2,
 1,
 4,
 1,
 3,
 4,
 2,
 4,
 2,
 2,
 2,
 5,
 2,
 2,
 3,
 2,
 3,
 1,
 4,
 5,
 3,
 6,
 2,
 3,
 3,
 4,
 3,
 5,
 3,
 4,
 4,
 6,
 0,
 4,
 2,
 4,
 3,
 2,
 2,
 2,
 3,
 2,
 3,
 4,
 2,
 4,
 5,
 1,
 3,
 3,
 4,
 2,
 4,
 5,
 1,
 3,
 2,
 2,
 3,
 3,
 0,
 6,
 6,
 6,
 4,
 3,
 2,
 4,
 2,
 5,
 1,
 4,
 7,
 2,
 3,
 3,
 5,
 4,
 5,
 5,
 2,
 7,
 2,
 0,
 2,
 3,
 3,
 3,
 4,
 1,
 4,
 2,
 4,
 3,
 2,
 5,
 2,
 3,
 1,
 2,
 5,
 3,
 5,
 0,
 3,
 5,
 5,
 4,
 3,
 2,
 3,
 6,
 5,
 3,
 2,
 6,
 3,
 5,
 5,
 4,
 3,
 6,
 2,
 2,
 8,
 2,
 3,
 6,
 3,
 4,
 3,


In [8]:
def check_uniqe_video_number_bothdirection(sub, ses):
    sessions_movies = read_settings_files_session(sub, ses)
    video_names = []
    for movies in sessions_movies:
        for movie in movies:
            video_names.append(movie.split("/")[-1].split(".")[0])
    videoset = set(video_names)
    print(videoset)
    print("unique video number: ", len(videoset))
    return videoset


def check_uniqe_video_number_uniqe_original(sub, ses):
    def desuffix(name):
        return name[:-3] #remove the suffix _fw or _rv

    sessions_movies = read_settings_files_session(sub, ses)
    video_names = []
    for movies in sessions_movies:
        for movie in movies:
            video_names.append(movie.split("/")[-1].split(".")[0])
    videoset = set(video_names)
    new_video_list = []
    for video in videoset:
        new_video_list.append(desuffix(video))

    videoset = set(new_video_list)
    print(
        "unique video number sub {sub} ses {ses}: ".format(sub=sub, ses=ses),
        len(videoset),
    )

    return videoset
    

In [9]:
for sub in range(1, subject_number + 1):
    for ses in range(1, session_number + 1):
        check_uniqe_video_number_bothdirection(sub, ses)

{'0012_fw', '0911_rv', '0464_fw', '1022_rv', '1217_fw', '0908_fw', '1634_rv', '2153_rv', '0049_fw', '1682_rv', '1747_rv', '1547_rv', '1783_rv', '1816_fw', '1643_rv', '1472_fw', '1930_fw', '0257_fw', '0060_fw', '1633_fw', '0587_rv', '0529_rv', '0147_rv', '0549_rv', '0585_rv', '0438_rv', '1078_fw', '1229_rv', '0031_rv', '0464_rv', '0252_fw', '0083_fw', '1217_rv', '1764_rv', '2037_fw', '1707_rv', '0356_rv', '0395_fw', '2034_rv', '1599_fw', '1744_rv', '1480_fw', '1731_fw', '0543_rv', '0477_rv', '0834_fw', '1880_fw', '1335_rv', '0433_fw', '2003_fw', '0425_fw', '0262_fw', '0862_fw', '0911_fw', '0438_fw', '0019_fw', '0031_fw', '1239_fw', '1568_rv', '1410_fw', '1400_fw', '0128_fw', '0585_fw', '1935_fw', '0305_rv', '1532_fw', '1131_fw', '1411_rv', '0418_rv', '0128_rv', '1894_fw', '0786_rv', '1289_rv', '0543_fw', '1815_rv', '1688_fw', '1972_rv', '1015_fw', '0649_fw', '2120_rv', '0309_fw', '1560_fw', '0740_rv', '1372_fw', '2048_fw', '1562_fw', '1440_rv', '0082_rv', '0851_rv', '1277_rv', '0802_fw'

In [10]:
for sub in range(1, subject_number + 1):
    for ses in range(1, session_number + 1):
        check_uniqe_video_number_uniqe_original(sub, ses)

unique video number sub 1 ses 1:  181
unique video number sub 1 ses 2:  181
unique video number sub 1 ses 3:  181
unique video number sub 1 ses 4:  181
unique video number sub 1 ses 5:  181
unique video number sub 1 ses 6:  181
unique video number sub 1 ses 7:  181
unique video number sub 1 ses 8:  181
unique video number sub 1 ses 9:  181
unique video number sub 1 ses 10:  181
unique video number sub 1 ses 11:  181
unique video number sub 1 ses 12:  181
unique video number sub 2 ses 1:  181
unique video number sub 2 ses 2:  181
unique video number sub 2 ses 3:  181
unique video number sub 2 ses 4:  181
unique video number sub 2 ses 5:  181
unique video number sub 2 ses 6:  181
unique video number sub 2 ses 7:  181
unique video number sub 2 ses 8:  181
unique video number sub 2 ses 9:  181
unique video number sub 2 ses 10:  181
unique video number sub 2 ses 11:  181
unique video number sub 2 ses 12:  181
unique video number sub 3 ses 1:  181
unique video number sub 3 ses 2:  181
unique

In [11]:
def session_unique_video_overlapping_check(sub, ses1, ses2):
    #check the overlapping of unique videos between different sessions
    videoset1 = check_uniqe_video_number_uniqe_original(sub, ses1)
    videoset2 = check_uniqe_video_number_uniqe_original(sub, ses2)
    print("overlapping: ", videoset1 & videoset2)

def all_session_unique_video_overlapping_check():
    for sub in range(1, subject_number + 1):
        for ses1 in range(1, session_number + 1):
            for ses2 in range(ses1 + 1, session_number + 1):
                session_unique_video_overlapping_check(sub, ses1, ses2)

In [12]:
all_session_unique_video_overlapping_check()

unique video number sub 1 ses 1:  181
unique video number sub 1 ses 2:  181
overlapping:  {'bl'}
unique video number sub 1 ses 1:  181
unique video number sub 1 ses 3:  181
overlapping:  {'bl'}
unique video number sub 1 ses 1:  181
unique video number sub 1 ses 4:  181
overlapping:  {'bl'}
unique video number sub 1 ses 1:  181
unique video number sub 1 ses 5:  181
overlapping:  {'bl'}
unique video number sub 1 ses 1:  181
unique video number sub 1 ses 6:  181
overlapping:  {'bl'}
unique video number sub 1 ses 1:  181
unique video number sub 1 ses 7:  181
overlapping:  {'bl'}
unique video number sub 1 ses 1:  181
unique video number sub 1 ses 8:  181
overlapping:  {'bl'}
unique video number sub 1 ses 1:  181
unique video number sub 1 ses 9:  181
overlapping:  {'bl'}
unique video number sub 1 ses 1:  181
unique video number sub 1 ses 10:  181
overlapping:  {'bl'}
unique video number sub 1 ses 1:  181
unique video number sub 1 ses 11:  181
overlapping:  {'bl'}
unique video number sub 1 se

In [13]:
def single_run_movie_repeat_count(subject,ses,run):
    settings = read_settings_file(subject, ses, run)
    movies = settings['stimuli']['movie_files']
    movies = [movie for movie in movies if movie != 'blank']
    count = 0
    last_movie = None
    for movie in movies:
        if movie == last_movie: 
            count += 1
        last_movie = movie
    return count

def all_run_movie_repeat_count():
    count_list = []
    for subject in range(1, subject_number+1):
        for ses in range(1, session_number+1):
            for run in range(1,run_number+1):
                count = single_run_movie_repeat_count(subject, ses, run)
                count_list.append(count)
                print(f"subject {subject}, session {ses} ,run {run} repeat count: {count}")
    return count_list

def single_run_movie_merge_direction_repeat_count(subject,ses,run):
    settings = read_settings_file(subject, ses, run)
    movies = settings['stimuli']['movie_files']
    movies = [movie for movie in movies if movie != 'blank']
    count = 0
    last_movie = None
    for movie in movies:
        if last_movie and movie[:-7] == last_movie[:-7]: 
            count += 1
        last_movie = movie
    return count

def all_run_movie_merge_direction_repeat_count():
    count_list = []
    for subject in range(1, subject_number+1):
        for ses in range(1, session_number+1):
            for run in range(1,run_number+1):
                count = single_run_movie_merge_direction_repeat_count(subject, ses, run)
                count_list.append(count)
                print(f"subject {subject}, session {ses} ,run {run} merge repeat count: {count}")
    return count_list
    

In [14]:
all_run_movie_repeat_count()

subject 1, session 1 ,run 1 repeat count: 0
subject 1, session 1 ,run 2 repeat count: 0
subject 1, session 1 ,run 3 repeat count: 0
subject 1, session 1 ,run 4 repeat count: 0
subject 1, session 1 ,run 5 repeat count: 0
subject 1, session 1 ,run 6 repeat count: 0
subject 1, session 1 ,run 7 repeat count: 0
subject 1, session 1 ,run 8 repeat count: 0
subject 1, session 1 ,run 9 repeat count: 0
subject 1, session 1 ,run 10 repeat count: 0
subject 1, session 2 ,run 1 repeat count: 0
subject 1, session 2 ,run 2 repeat count: 0
subject 1, session 2 ,run 3 repeat count: 0
subject 1, session 2 ,run 4 repeat count: 0
subject 1, session 2 ,run 5 repeat count: 0
subject 1, session 2 ,run 6 repeat count: 0
subject 1, session 2 ,run 7 repeat count: 0
subject 1, session 2 ,run 8 repeat count: 0
subject 1, session 2 ,run 9 repeat count: 0
subject 1, session 2 ,run 10 repeat count: 0
subject 1, session 3 ,run 1 repeat count: 0
subject 1, session 3 ,run 2 repeat count: 0
subject 1, session 3 ,run 3 re

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [15]:
all_run_movie_merge_direction_repeat_count()

subject 1, session 1 ,run 1 merge repeat count: 0
subject 1, session 1 ,run 2 merge repeat count: 1
subject 1, session 1 ,run 3 merge repeat count: 0
subject 1, session 1 ,run 4 merge repeat count: 0
subject 1, session 1 ,run 5 merge repeat count: 0
subject 1, session 1 ,run 6 merge repeat count: 0
subject 1, session 1 ,run 7 merge repeat count: 0
subject 1, session 1 ,run 8 merge repeat count: 0
subject 1, session 1 ,run 9 merge repeat count: 0
subject 1, session 1 ,run 10 merge repeat count: 0
subject 1, session 2 ,run 1 merge repeat count: 0
subject 1, session 2 ,run 2 merge repeat count: 0
subject 1, session 2 ,run 3 merge repeat count: 1
subject 1, session 2 ,run 4 merge repeat count: 0
subject 1, session 2 ,run 5 merge repeat count: 0
subject 1, session 2 ,run 6 merge repeat count: 0
subject 1, session 2 ,run 7 merge repeat count: 0
subject 1, session 2 ,run 8 merge repeat count: 0
subject 1, session 2 ,run 9 merge repeat count: 0
subject 1, session 2 ,run 10 merge repeat count: 

[0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 2,
 0,
 0,
 2,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 2,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
